In [ ]:
from ai_council.council import *
from ai_council.prompts import *
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import PydanticOutputParser
import ast

[(k["llm"] , k["name"]) for k in MODELS]

In [ ]:
# responses = []
# for i,k in enumerate(MODELS):
#     if k['id'] == 'evaluator':
#         continue
#     print(f"Response by {k["name"]}")
#     text = k["llm"].invoke('Tell me a joke')
#     print(text)
#     print("____________________________________________________")
#     responses.append(
#         {
#             "response_id" : f"r_{i}",
#             "model_id" : k["id"],
#             "text" : text
#         }
#     )

In [ ]:
# scorer_parser = PydanticOutputParser(pydantic_object=scoring_output)
# scoring_prompt = ChatPromptTemplate.from_template(scoring_template)
# chains = [(scoring_prompt|k["llm"], k) for k in MODELS]
# scoring_matrix = {}
# for chain in chains:
#     temp_dict = {}
#     print(chain[1]['name'])
#     for i in range(len(responses)):
#         result = chain[0].invoke({"user_prompt" : "Tell me a joke", "candidate_response" : responses[i], "output_format" : scorer_parser.get_format_instructions()})
#         print(result)
#         json_response = ast.literal_eval('{' + extract_first_curly_balanced(result) + '}')
#         json_response['total'] = sum([WEIGHTS[k] * json_response['scores'][k] for k in WEIGHTS])
#         temp_dict[responses[i]["response_id"]] = json_response
#     scoring_matrix[chain[1]['name']] = temp_dict

In [ ]:
responses , user_prompt = generate_expert_response("Who was the first female mountaineer to climb Mount Everest", "")
for response in responses:
    print(response)

In [ ]:
scoring_matrix = generate_scores(responses, user_prompt)

In [ ]:
user_prompt

In [ ]:
# import re
# ast.literal_eval('{'+re.sub(r'//.*','',extract_first_curly_balanced(scoring_results[-1])).replace('null' , '0') + '}')

In [ ]:
pretty_print_json(scoring_matrix)

In [ ]:
audit = generate_audit_report(user_prompt, responses, scoring_matrix)

In [ ]:
# from pydantic import BaseModel
# from typing import List, Optional, Dict
# from ai_council.constants import MODELS


# ##### Scoring Output Format
# class Scores(BaseModel):
#     accuracy : int
#     completeness : int
#     grounding : int
#     reasoning : int
#     clarity : int

# class scoring_output(BaseModel):
#     scores : Scores
#     confidence_estimate : float
#     justification : str


# ##### Audit Output Format
# class flag(BaseModel):
#     scorer_id : str
#     issue : str
#     severity : str
# class Audit_Report(BaseModel):
#     audit_id : str
#     flags : List[flag]
#     drops : List[str]
#     explanation : str
#     normalization : Dict[str, float]


# scoring_template ="""
#     SYSTEM: You are an impartial evaluator that scores candidate answers to a user prompt. Use the rubric provided and be objective. 
#     Return only the JSON object described below and nothing else.

#     USER: Here is the ORIGINAL USER PROMPT:
#     {user_prompt}

#     Here is the CANDIDATE RESPONSE you must evaluate:
#     {candidate_response}

#     RUBRIC (score each 1-5; 5 = best):
#     - accuracy: Is the content factually correct given known, verifiable facts? (1 = many factual errors or hallucinations; 5 = fully accurate)
#     - completeness: Does it address all parts of the prompt? (1 = misses core parts; 5 = full coverage)
#     - grounding: Does the response cite or reference verifiable sources or show evidence/reasoning that can be checked? (1 = unsupported claims; 5 = well-grounded)
#     - reasoning: Are the logical steps coherent and correct? (1 = flawed reasoning; 5 = sound stepwise logic)
#     - clarity: Is it readable, appropriately toned, and well-structured? (1 = confusing; 5 = clear & concise)

#     Also provide a one-sentence justification for the total score and a confidence estimate between 0 and 1.

#     Output Format : 
#     {output_format}

#     Notes:
#     - Score numerically and be conservative: penalize minor hallucinations or unsupported numeric claims.
#     - Do not refer to model names, internals, or policies in your justification.

# """

# expert_generation_template = """
#     SYSTEM:
#     You are a retrieval-grounded assistant. Use only the information in the CONTEXT. 
#     If the answer is not in the context, say: "Not in context." 
#     Do not guess or invent facts.

#     FORMAT:
#     1. Final answer (1-2 lines)
#     2. Brief reasoning (1-2 lines)
#     3. Snippets used (# or "none")

#     USER:
#     {user_prompt}

#     CONTEXT:
#     {context}

#     RULES:
#     - Base all statements strictly on the context.
#     - Cite snippet numbers when used.
#     - Keep responses short and precise.
# """


# auditor_prompt_template_1 = """
#     SYSTEM: You are an independent auditor whose job is to inspect a scoring matrix produced by peer models and detect bias, collusion, or anomalous scoring patterns. Return only the JSON described below.

#     USER: We provide:
#     1) original_prompt: {user_prompt}
#     2) responses: a JSON list of response objects:
#     {responses}
#     3) scoring_matrix: a JSON object where keys are scorer_ids and values are dictionaries mapping response_id -> score_obj
#     e.g., {scoring_matrix}

#     Task:
#     1) Inspect scoring patterns for the following anomalies:
#     - Self-scoring or allowed self-favoring (scorer giving systematically higher scores to a single partner)
#     - Collusion: two or more scorers consistently upvoting each other across many prompts (pattern detection)
#     - Extreme scorers: scorer that always gives very high (>=4.5) or very low (<=1.5) totals while variance is near zero
#     - Outliers: scorer scores that deviate > 2 std from the mean for a response

#     2) For each detected anomaly produce a corrective action:
#     - normalization factor for that scorer (multiply all their scores by that factor)
#     - or flag for human review (if severe)
#     - or drop scorer from aggregation for this prompt

#     Return JSON **only**:
#     {output_format}
# """


In [ ]:
# import time, json, statistics, ast, re
# import numpy as np
# # from ai_council.generate_prompts import *
# from ai_council.prompts import *
# from ai_council.constants import *
# from langchain_ollama import OllamaLLM
# from langchain_core.prompts import ChatPromptTemplate
# from langchain_core.output_parsers import PydanticOutputParser

# scoring_results = []

# def extract_first_curly_balanced(text):
#     """
#     Extracts the substring between the first balanced set of curly braces {}.
#     Handles nested braces correctly.
#     Returns None if no valid balanced braces are found.
#     """
#     if not isinstance(text, str):
#         raise TypeError("Input must be a string.")

#     start_index = None
#     brace_count = 0

#     for i, char in enumerate(text):
#         if char == '{':
#             if brace_count == 0:
#                 start_index = i + 1  # content starts after '{'
#             brace_count += 1
#         elif char == '}':
#             brace_count -= 1
#             if brace_count == 0 and start_index is not None:
#                 return text[start_index:i]  # return content inside braces
#             if brace_count < 0:
#                 # Found closing brace before opening
#                 return None

#     return None  # No balanced braces found



# def pretty_print_json(json_data):
#     """
#     Pretty prints JSON data with indentation and sorted keys.
#     Accepts either a Python dict/list or a JSON string.
#     """
#     try:
#         # If input is a string, parse it to Python object
#         if isinstance(json_data, str):
#             json_obj = json.loads(json_data)
#         elif isinstance(json_data, (dict, list)):
#             json_obj = json_data
#         else:
#             raise TypeError("Input must be a JSON string, dict, or list.")

#         # Pretty print with indentation and sorted keys
#         pretty_json = json.dumps(json_obj, indent=4, sort_keys=True, ensure_ascii=False)
#         print(pretty_json)

#     except json.JSONDecodeError as e:
#         print(f"Invalid JSON string: {e}")
#     except Exception as e:
#         print(f"Error: {e}")


# def generate_scores(responses, user_prompt, llm = None):
#     scorer_parser = PydanticOutputParser(pydantic_object=scoring_output)
#     scoring_prompt = ChatPromptTemplate.from_template(scoring_template)
#     chains = [(scoring_prompt|k["llm"], k) for k in MODELS]
#     scoring_matrix = {}
#     for chain in chains:
#         if chain[1]['id'] == "evaluator":
#             continue
#         if llm and chain[1]['name'] != llm:
#             continue
#         temp_dict = {}
#         print(chain[1]['name'])
#         for i in range(len(responses)):
#             result = chain[0].invoke({"user_prompt" : user_prompt, "candidate_response" : responses[i], "output_format" : scorer_parser.get_format_instructions()})
#             print(f"Scoring complete for {responses[i]['response_id']} by {chain[1]['name']}")
#             scoring_results.append(result)
#             json_response = ast.literal_eval('{'+re.sub(r'//.*','',extract_first_curly_balanced(scoring_results[-1])).replace('null' , '0') + '}')
#             json_response['total'] = sum([WEIGHTS[k] * json_response['scores'][k] for k in WEIGHTS])
#             temp_dict[responses[i]["response_id"]] = json_response
#         scoring_matrix[chain[1]['name']] = temp_dict
#     return scoring_matrix

# def generate_expert_response(user_prompt, context):
#     prompt = ChatPromptTemplate.from_template(expert_generation_template)
#     return_prompt = prompt.invoke({"user_prompt" : user_prompt, "context" : context})
#     chains = [(prompt|k['llm'] , k) for k in MODELS]
#     responses = []
#     for i, chain in enumerate(chains):
#         if chain[1]['id'] == "evaluator":
#             continue
#         print(f"Response from {chain[1]['name']}")
#         result = chain[0].invoke({"user_prompt" : user_prompt, "context" : context})
#         responses.append(
#             {
#                 "response_id" : f"r_{i}",
#                 "model_id" : chain[1]["id"],
#                 "text" : result
#             }
#         )
#         print(f"Response generated by {chain[1]['name']} with id r_{i}")
#     return responses, return_prompt

# def generate_audit_report(user_prompt, responses, scoring_matrix):
#     audit_report = PydanticOutputParser(pydantic_object=Audit_Report)
#     audit_prompt = ChatPromptTemplate.from_template(auditor_prompt_template_1)
#     auditor =[k for k in MODELS if k["id"] == "evaluator"]
#     chain = audit_prompt|auditor[0]['llm']
#     result = chain.invoke({"user_prompt" : user_prompt, "responses" : responses, "scoring_matrix" : scoring_matrix, "output_format": audit_report.get_format_instructions()})
#     return result

In [ ]:
audit = generate_audit_report(user_prompt, responses, scoring_matrix)

In [ ]:
print(pretty_print_json(audit))

In [ ]:
audit_1 = {
    "audit_id": "audit_001",
    "drops": [
        "phi4",
        "gemma2:9b"
    ],
    "explanation": "Across the five responses for this prompt, scorer_id \"phi3:mini\" provided a total of 2.45 for response r_0, which is an outlier relative to the mean total of 4.61 (σ≈0.89) for the rest of the cohort, and the scorer also omitted required completion and reasoning fields. The other two extreme scorers, \"phi3:mini\" and \"gemma2:9b\", were evaluated as giving systematically high scores (>=4.5) with almost no variance; therefore they are dropped from the aggregation for this prompt. No self‑favoring or collusion patterns were detected in this single‑prompt data set. No normalization factors are applied.",
    "flags": [
        {
            "issue": "Non‑uniform total scores and incomplete evaluation (high accuracy but low completeness, no grounding, no reasoning).",
            "scorer_id": "phi3:mini",
            "severity": "moderate"
        },
        {
            "issue": "Deviation from expected score distribution on this prompt.",
            "scorer_id": "phi3:mini",
            "severity": "moderate"
        }
    ],
    "normalization": {}
}

In [ ]:
print(audit)

In [ ]:
print(scoring_results[-2])

In [ ]:
audit